In [ ]:
# TABLE ONE

In [94]:
#Required python libraries

In [28]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [29]:
# driver and browser
url ='https://www.zomato.com/bangalore/'
driver = webdriver.Firefox()
driver.get(url)


In [30]:
# Clicking " see more"  button.
button = driver.find_elements(By.XPATH, "//div[contains(text(),'see more')]")
button[0].click()

In [31]:
# defining BeautifulSoup library
soup = BeautifulSoup(driver.page_source,"html.parser")

In [32]:
# collection all the location LINKS by using Beautifulsoup in list called LOCATIONLIST
locationlist = []
for link in soup.find_all('a', href = True):
    if 'https://www.zomato.com/bangalore/' in link['href']:
        locationlist.append(link['href'])
print(len(locationlist))

29


In [1]:
# getting all types of link  from all the links in locationlist
link = []
for i in range(len(locationlist)):
    driver.get(locationlist[i])
    time.sleep(1)  # Allow 2 seconds for the web page to (open depends on you)
    scroll_pause_time = 2 # You can set your own pause time. dont slow too slow that might not able to load more data
    screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
    A = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{A});".format(screen_height=screen_height, A=A))
        A += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * A > scroll_height:
            break

    links = driver.find_elements('tag name','a')
    for i in links:
        link.append(i.get_attribute('href'))

In [41]:
# separating Restaurant links from all the links in list called LIST
Restaurant_link=[]
for i in link:
    try:
        if 'order' in i:
            Restaurant_link.append(i)
    except:
        pass

In [3]:
# deleting the reaping link by make the list to set
final_Restaurant_link = set(Restaurant_link)
print(len(final_Restaurant_link))

In [44]:
# making it list from set
Restaurants=[]
for i in final_Restaurant_link:
    try:
        if 'order' in i:
            Restaurants.append(i)
    except:
        pass

In [ ]:
# price_for_one
price_for_one = []
count = 0
remove_order = [i.replace('/order','') for i in Restaurants['link']]
for i in range(len(remove_order)):
    count+=1
    print (count, end =" ")
    driver.get(remove_order[i])
    try:
        price = driver. find_element (By.XPATH, "//p[contains(text(), '₹')]")
        price_for_one.append (price.text)
    except:
        price_for_one.append (np.nan)

In [46]:
# adding values in different types of atributes
name=[]
rating=[]
cusines=[]
location=[]
delivery_review_number=[]
count = 0
for i in Restaurants:
    count+=1
    print(count,end=" ")
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    sou = BeautifulSoup(r, 'lxml')
    try:
        j = sou.find_all("h1",{"class" : "sc-7kepeu-0 sc-eilVRo eAhpQG"})
        name.append(j[0].text)
    except:
        name.append('NA')
    try:
        j = sou.find_all("div",{"class" : "sc-1q7bklc-1 cILgox"})
        rating.append(j[1].text)
    except:
        rating.append('NA')
    
    try:
        j = sou.find_all("div",{"class" : "sc-eMigcr fAGAHS"})
        cusines.append(j[0].text)
    except:
        cusines.append('NA')
    try:
        j = sou.find_all("a",{"class" : "sc-cpmLhU fDVcNc"})
        location.append(j[0].text)
    except:
        location.append('NA')

    try:
        j = sou.find_all("div",{"class" : "sc-1q7bklc-8 kEgyiI"})
        delivery_review_number.append(j[1].text)
    except:
        delivery_review_number.append('NA')
    

    
    

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 

In [4]:
# converting it in data frame
col=['name','rating','cusines','delivery_review_number','location','price_for_one']
df=pd.DataFrame({'Name': name,'Rating':rating,'cusines':cusines,'delivery_review_number':delivery_review_number,'location':location,'price_for_one':price_for_one})


In [86]:
#  adding longitude and latitude of different locations to data frame
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(location): 
    try:
        geolocator = Nominatim(user_agent="chrome")
          
        return geolocator.geocode(location)
      
    except GeocoderTimedOut:
          
        return findGeocode(location)  
for i in (df["location"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

df["Longitude"] = longitude
df["Latitude"] = latitude



In [8]:
# adding all  the location links to data frame
df["Restaurant_link"]  = Restaurants
df

In [9]:
# converting dataframe in csv and downloading it.
df.to_csv(r"C:\Users\Vaishali Tomar\Desktop\zomato_project\restaurant_info.csv")

In [ ]:
# TABLE TWO

In [10]:
restaurant_name = []
dish_name = []
dish_price = []
location=[]
count = 0

for i in Restaurants :
    count+=1
    print(count,end=" ")
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    sou = BeautifulSoup(r, 'lxml')
    for i in range(10):
        try:
            j = sou.find_all("h4",{"class" : "sc-1s0saks-15 iSmBPS"})
            dish_name.append(j[i].text)
        except:
            dish_name.append('NA')
        try:
            j = sou.find_all("span",{"class" : "sc-17hyc2s-1 cCiQWA"})
            dish_price.append(j[i].text)
        except:
            dish_price.append('NA')
        try:
            j = sou.find_all("h1",{"class" : "sc-7kepeu-0 sc-eilVRo eAhpQG"})#sc-7kepeu-0 sc-eilVRo eAhpQG
            restaurant_name.append(j[0].text)
        except:
            restaurant_name.append('NA')
        try:
            j = sou.find_all("a",{"class" : "sc-cpmLhU fDVcNc"})
            location.append(j[0].text)
        except:
            location.append('NA')
col=['location','restaurant_name','dish_name','cusines','dish_price']
df1=pd.DataFrame({'location':location,'restaurant_name': restaurant_name,'dish_name':dish_name,'dish_price':dish_price})
df1.to_csv(r"C:\Users\Vaishali Tomar\Desktop\zomato_project\restaurant_details")

In [11]:
# converting it to data frame
col=['restaurant_name','dish_name','cusines','dish_price']
df1=pd.DataFrame({'restaurant_name': restaurant_name,'dish_name':dish_name,'dish_price':dish_price})
df1

In [93]:
# downloading and converting it to csv
df1.to_csv(r"C:\Users\Vaishali Tomar\Desktop\zomato_project\restaurant_details.csv")